In [1]:
from utils import *
from sklearn.datasets import make_friedman1, make_friedman2, make_friedman3
from scipy import optimize
from scipy.stats import spearmanr
from multiprocessing import Pool, cpu_count, set_start_method
from functools import partial
from itertools import repeat
import pandas as pd
import os
depths = (3, 100)
# corrcoef_type = 'p'
# c_val_bounds = 'fixed' #(1e-9, 1e5) 

In [2]:
def runner(
    fried_num, 
    noise, 
    norm,
    c_val_bounds, 
    ntk_depth, 
    ansiotropic,
    white,
    w_fixed=False,
    alpha=False,
    opt_bounds = [0.0001, 10],
    opt_resets = 10,
    figs = False,
    diagnostic = False,
    exp_name = ''
    ):

    if type(ntk_depth) == int:
        ntk_depth = (ntk_depth,)

    return_vals = {
        'Dataset' : f'Friedman {fried_num}',
        'Noise': False if noise == 0.0 else True,
        'Norm' : norm,
        'Constant Value Bounds' : 'fixed' if c_val_bounds == 'fixed' else 'unfixed',
        'White/Bounds/Alpha' : (white, w_fixed, alpha)
        }
    print(
        f"Friedman {fried_num}, Noise: {noise}, Norm: {norm}, cv_bounds: {c_val_bounds}, white/bounds/alpha: {white, w_fixed, alpha}"
    )

    # dir level:
    # exp
    #   Dataset
    #       plots

    job_name = f"Pred_N{noise}_N_{norm}_c{return_vals['Constant Value Bounds']}_ans_{ansiotropic}_w_{white}_w_fixed_{w_fixed}_alp_{alpha}"

    exp_name = os.path.join(os.getcwd(), 'experiments', exp_name, f'F{fried_num}')
    if not os.path.isdir(exp_name):
        os.makedirs(exp_name)

    opt_lower, opt_upper = opt_bounds

    # Datasets
    X_init, y_init = [None, None]
    if fried_num==1:
        X_init, y_init = make_friedman1(noise=noise, random_state=18397425)
    elif fried_num==2:
        X_init, y_init = make_friedman2(noise=noise, random_state=18397425)
    elif fried_num==3:
        X_init, y_init = make_friedman3(noise=noise, random_state=18397425)
    else:
        return Exception("fried_num must be either 1, 2, or 3")
    
    datasets = processing(X_init, y_init)
    X, y, X_train, y_train = [None, datasets['orig'][1], None, datasets['orig train'][1]]

    if norm:
        X = datasets['norm'][0]
        X_train = datasets['norm train'][0]
    else:
        X = datasets['orig'][0]
        X_train = datasets['orig train'][0]



    # Lengthscale Optimization
    def g(ell, gp, mean_ntk, typ):
        try:
            gp.set_params(**{'kernel__k2__length_scale': ell})
        except:
            gp.set_params(**{'kernel__k1__k2__length_scale': ell})

        gp.fit(X_train, y_train)
        mean = gp.predict(X)

        if diagnostic == True:
            print(typ, gp.kernel_)

        return np.sum((mean_ntk - mean)**2)

    for depth in ntk_depth:
        # Kernels
        if noise != 0.0 and white == True and alpha == False:
            ntk = (
                ConstantKernel(
                    constant_value=1,
                    constant_value_bounds=c_val_bounds
                ) *
                NTK(depth=depth, c=2,
                    bias=1e-1,
                    bias_bounds=(1e-9, 1e3)
                ) + 
                WhiteKernel(
                    noise_level=0.15**2, 
                    noise_level_bounds='fixed' if w_fixed else (1e-4, 0.5))#(0.001, 1))
            )
        else:
            ntk = (
                ConstantKernel(
                    constant_value=1,
                    constant_value_bounds=c_val_bounds#(1e-9, 1e2)
                ) *
                NTK(depth=depth, c=2,
                    bias=1e-1,
                    bias_bounds=(1e-9, 1e3)
                )
            )

        gp_ntk = GPR(kernel=ntk, alpha=1e-9 if not alpha else 0.15**2, normalize_y=True,
                    n_restarts_optimizer=9, random_state=3480795)
        gp_ntk.fit(X_train, y_train)
        print(f'NTK {depth} : ', gp_ntk.kernel_)
        mean_ntk = gp_ntk.predict(X)
        
        # print(gp_ntk.kernel_.get_params())


        if noise != 0.0 and white == True:
            lpk = (
                ConstantKernel(
                    constant_value=gp_ntk.kernel_.get_params()['k1__k1__constant_value'],
                    constant_value_bounds='fixed'
                ) *
                Matern(
                    nu=1/2,
                    length_scale=np.array([np.random.rand()*np.random.randint(1, 1000) 
                                        for i in range(0, X.shape[1])]) if ansiotropic else 1,
                    length_scale_bounds='fixed'
                ) + 
                WhiteKernel(
                    noise_level=gp_ntk.kernel_.get_params()['k2__noise_level'], 
                    noise_level_bounds='fixed')
            )
        else:
            lpk = (
                ConstantKernel(
                    constant_value=gp_ntk.kernel_.get_params()['k1__constant_value'],
                    constant_value_bounds='fixed'
                ) *
                Matern(
                    nu=1/2,
                    # anisotropic true -> trying to fit product of laplace kernels with different lengthscales
                    length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                        for i in range(0, X.shape[1])]) if ansiotropic else 1,
                    length_scale_bounds='fixed'
                )
            )

        gp_lpk = GPR(kernel=lpk, alpha=1e-9 if not alpha else 0.15**2, normalize_y=True,
                    n_restarts_optimizer=0, random_state=3480795)

        if ansiotropic:
            ell_lpk = optimize.minimize(g, np.array([np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]), args=(
                gp_lpk, mean_ntk, f'LPK {depth}:'), 
                bounds=tuple((0.00001, 10) for n in range(0, X.shape[1])))
            for i in range(0, opt_resets):
                tmp = optimize.minimize(g, np.array([[np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]]), args=(
                    gp_lpk, mean_ntk, f'LPK {depth}:'), 
                    bounds=tuple((0.00001, 10**i) for n in range(0, X.shape[1])))
                if tmp.fun < ell_lpk.fun:
                    ell_lpk = tmp

        else:
            ell_lpk = optimize.minimize_scalar(g, args=(
                gp_lpk, mean_ntk, f'LPK {depth}:'), 
                method='bounded', bounds=opt_bounds, options={'maxiter': 10000})
            for i in range(0, opt_resets):
                tmp = optimize.minimize_scalar(g, args=(
                    gp_lpk, mean_ntk, f'LPK {depth}:'),
                    method='bounded', bounds=[opt_lower, opt_upper*(10**i)], options={'maxiter': 10000})
                if tmp.fun < ell_lpk.fun:
                    ell_lpk = tmp

        try:
            gp_lpk.set_params(**{'kernel__k2__length_scale': ell_lpk.x})
        except:
            gp_lpk.set_params(**{'kernel__k1__k2__length_scale': ell_lpk.x})
        gp_lpk.fit(X_train, y_train)
        mean_lpk_opt = gp_lpk.predict(X)


        #################################
        # Begin gathering return values #
        #################################
        return_vals[f'x_lpk_{depth}'] = ell_lpk.x
        return_vals[f'f_lpk_{depth}'] = ell_lpk.fun
        return_vals[f'Pcorr_lpk_{depth}_res'] = np.corrcoef((y-mean_ntk)[:,0], (y-mean_lpk_opt)[:,0])[0, 1]
        return_vals[f'Pcorr_lpk_{depth}_y'] = np.corrcoef((mean_ntk)[:,0], (mean_lpk_opt)[:,0])[0, 1]
        corr_lpk_res = return_vals[f'Pcorr_lpk_{depth}_res']
        corr_lpk_y   = return_vals[f'Pcorr_lpk_{depth}_y']



        if norm:
            if noise != 0.0 and white == True and alpha == False:
                rbf = (
                    ConstantKernel(
                        constant_value=1, 
                        constant_value_bounds=c_val_bounds
                    ) * 
                    Matern(
                        nu=np.inf,
                        length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                               for i in range(0, X.shape[1])]) if ansiotropic else 1,
                        length_scale_bounds='fixed'
                    ) + 
                    WhiteKernel(
                        noise_level=gp_ntk.kernel_.get_params()['k2__noise_level'],
                        noise_level_bounds='fixed'
                    )
                )
            else:
                rbf = (
                    ConstantKernel(
                        constant_value=1,
                        constant_value_bounds=c_val_bounds
                    ) *
                    Matern(
                        nu=np.inf,
                        length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                               for i in range(0, X.shape[1])]) if ansiotropic else 1,
                        length_scale_bounds='fixed'
                    )
                )

            gp_rbf = GPR(kernel=rbf, alpha=1e-9 if not alpha else 0.15**2, normalize_y=True, n_restarts_optimizer=0, random_state=3480795)

            if ansiotropic:
                ell_rbf = optimize.minimize(g, np.array([np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]), args=(
                    gp_rbf, mean_ntk, f'RBF {depth}:'), 
                    bounds=tuple((0.00001, 10) for n in range(0, X.shape[1])))
                for i in range(0, opt_resets):
                    tmp = optimize.minimize(g, np.array([[np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]]), args=(
                        gp_rbf, mean_ntk, f'RBF {depth}:'), 
                        bounds=tuple((0.00001, 10**i) for n in range(0, X.shape[1])))
                    if tmp.fun < ell_rbf.fun:
                        ell_rbf = tmp

            else:
                ell_rbf = optimize.minimize_scalar(g, args=(
                    gp_rbf, mean_ntk, f'RBF {depth}:'), 
                    method='bounded', bounds=opt_bounds, options={'maxiter': 10000})
                for i in range(0, opt_resets):
                    tmp = optimize.minimize_scalar(g, args=(
                        gp_rbf, mean_ntk, f'RBF {depth}:'),
                        method='bounded', bounds=[opt_lower, opt_upper*(10**i)], options={'maxiter': 10000})
                    if tmp.fun < ell_rbf.fun:
                        ell_rbf = tmp

            try:
                gp_rbf.set_params(**{'kernel__k2__length_scale': ell_rbf.x})
            except:
                gp_rbf.set_params(**{'kernel__k1__k2__length_scale': ell_rbf.x})
            gp_rbf.fit(X_train, y_train)
            mean_rbf_opt = gp_rbf.predict(X)



            ####################################
            # Continue gathering return values #
            ####################################
            return_vals[f'x_rbf_{depth}'] = ell_rbf.x
            return_vals[f'f_rbf_{depth}'] = ell_rbf.fun
            return_vals[f'Pcorr_rbf_{depth}_res'] = np.corrcoef((y-mean_ntk)[:,0], (y-mean_rbf_opt)[:,0])[0, 1]
            return_vals[f'Pcorr_rbf_{depth}_y'] = np.corrcoef((mean_ntk)[:,0], (mean_rbf_opt)[:,0])[0, 1]
            corr_rbf_res = return_vals[f'Pcorr_rbf_{depth}_res']
            corr_rbf_y = return_vals[f'Pcorr_rbf_{depth}_y']



        if figs:
            print("Creating plots...")
            plt.ioff()
            rvr = os.path.join(exp_name, 'res_vs_res')
            rvy = os.path.join(exp_name, 'res_vs_y')
            yvy = os.path.join(exp_name, 'y_vs_y')
            if not os.path.isdir(rvr):
                os.makedirs(rvr)
            if not os.path.isdir(rvy):
                os.makedirs(rvy)
            if not os.path.isdir(yvy):
                os.makedirs(yvy)

            # residuals vs y_hat (LPK and NTK SEPARATE)
            fig, axes = plt.subplots(nrows=1, ncols=3 if norm else 2, sharex=True,sharey=True, figsize=(12 if norm else 8, 4))
            axes[0].scatter(y-mean_ntk, mean_ntk)
            axes[0].set_title('NTK')
            axes[1].scatter(y-mean_lpk_opt, mean_lpk_opt)
            axes[1].set_title('LPK')
            title = f"{gp_ntk.kernel_}\n{gp_lpk.kernel_}"
            if norm:
                axes[2].scatter(y-mean_rbf_opt, mean_rbf_opt)
                axes[2].set_title('RBF')
                title += f"\n{gp_rbf.kernel_}"
            fig.supxlabel('Residuals')
            fig.supylabel(r'$\hat{y}$')
            fig.suptitle(r"Residuals vs. $\hat{y}$ for " + f"NTK Depth = {depth}\n" + title)
            fig.tight_layout()
            fig.savefig(f'{rvy}/depth_{depth}_{job_name}.png', dpi=150, transparent=False, bbox_inches='tight')
            
            # residuals vs residuals (LPK vs NTK)
            fig, axes = plt.subplots(nrows=1, ncols=2 if norm else 1, sharex=True, sharey=True, figsize=(8 if norm else 4, 4))
            if norm:
                axes[0].scatter(y-mean_lpk_opt, y-mean_ntk)
                axes[0].set_xlabel(f"LPK Residuals\nCorr: {round(corr_lpk_res, 4)}")
                axes[1].scatter(y-mean_rbf_opt, y-mean_ntk)
                axes[1].set_xlabel(f"RBF Residuals\nCorr: {round(corr_rbf_res, 4)}")
            else:
                axes.scatter(y-mean_lpk_opt, y-mean_ntk)
                axes.set_xlabel(f"LPK Residuals\nCorr: {round(corr_lpk_res, 4)}")
            fig.supylabel("NTK Residuals")
            fig.suptitle(f"Residuals vs. Residuals for NTK Depth = {depth}\n" + title)
            fig.tight_layout()
            fig.savefig(f'{rvr}/depth_{depth}_{job_name}.png', dpi=150, transparent=False, bbox_inches='tight')

            # y_hat vs y_hat (LPK vs NTK)
            fig, axes = plt.subplots(nrows=1, ncols=2 if norm else 1, sharex=True, sharey=True, figsize=(8 if norm else 4, 4))
            if norm:
                axes[0].scatter(mean_lpk_opt, mean_ntk)
                axes[0].set_xlabel(r'$y_{LPK}$' + f"\nCorr: {round(corr_lpk_y, 4)}")
                axes[1].scatter(mean_rbf_opt, mean_ntk)
                axes[1].set_xlabel(r'$y_{RBF}$' + f"\nCorr: {round(corr_rbf_y, 4)}")
            else:
                axes.scatter(mean_lpk_opt, mean_ntk)
                axes.set_xlabel(r'$y_{LPK}$' + f"\nCorr: {round(corr_lpk_y, 4)}")
            fig.supylabel(r'$y_{NTK}$')
            fig.suptitle(f"Predicted mean comparison for NTK Depth = {depth}\n" + title)
            fig.tight_layout()
            fig.savefig(f'{yvy}/depth_{depth}_{job_name}.png', dpi=150, transparent=False, bbox_inches='tight')
            plt.ion()

    return return_vals

In [22]:
def runner(
    fried_num, 
    noise, 
    norm,
    c_val_bounds, 
    corrcoef_type,
    ntk_depth, 
    ansiotropic,
    white,
    w_fixed=False,
    alpha=False,
    opt_corr=True,
    opt_bounds = [0.0001, 10],
    opt_resets = 10,
    figs = False,
    diagnostic = False,
    exp_name = ''
    ):

    if type(ntk_depth) == int:
        ntk_depth = (ntk_depth,)

    return_vals = {
        'Dataset' : f'Friedman {fried_num}',
        'Noise': False if noise == 0.0 else True,
        'Norm' : norm,
        'Constant Value Bounds' : 'fixed' if c_val_bounds == 'fixed' else 'unfixed',
        'Corr Type' : 'Pearson ' if corrcoef_type == 'p' else 'Spearman '
        }
    print(
        f"Friedman {fried_num}, Noise: {noise}, Norm: {norm}, cv_bounds: {c_val_bounds}, corrcoef_type: {corrcoef_type}"
    )

    # dir level:
    # exp
    #   Dataset
    #       plots

    job_name = f"N{noise}_N_{norm}_c{return_vals['Constant Value Bounds']}_{corrcoef_type}_ans_{ansiotropic}_w_{white}_w_fixed_{w_fixed}"

    exp_name = os.path.join(os.getcwd(), 'experiments', exp_name, f'F{fried_num}')
    if not os.path.isdir(exp_name):
        os.makedirs(exp_name)

    opt_lower, opt_upper = opt_bounds

    # Datasets
    X_init, y_init = [None, None]
    if fried_num==1:
        X_init, y_init = make_friedman1(noise=noise, random_state=18397425)
    elif fried_num==2:
        X_init, y_init = make_friedman2(noise=noise, random_state=18397425)
    elif fried_num==3:
        X_init, y_init = make_friedman3(noise=noise, random_state=18397425)
    else:
        return Exception("fried_num must be either 1, 2, or 3")
    
    datasets = processing(X_init, y_init)
    X, y, X_train, y_train = [None, datasets['orig'][1], None, datasets['orig train'][1]]

    if norm:
        X = datasets['norm'][0]
        X_train = datasets['norm train'][0]
    else:
        X = datasets['orig'][0]
        X_train = datasets['orig train'][0]



    # Lengthscale Optimization
    if opt_corr:
        def g(ell, gp, input, typ, p_or_s='p', res=False):
            try:
                gp.set_params(**{'kernel__k2__length_scale': ell})
            except:
                gp.set_params(**{'kernel__k1__k2__length_scale': ell})

            gp.fit(X_train, y_train)
            mean = gp.predict(X)

            if diagnostic == True:
                print(typ, gp.kernel_)

            if p_or_s == 's':
                return -spearmanr(input, mean if not res else y-mean)[0]
            else:
                return -np.corrcoef((input)[:, 0], (mean if not res else y-mean)[:, 0])[0, 1]
    else:
        def g(ell, gp, mean_ntk, typ, _):
            try:
                gp.set_params(**{'kernel__k2__length_scale': ell})
            except:
                gp.set_params(**{'kernel__k1__k2__length_scale': ell})

            gp.fit(X_train, y_train)
            mean = gp.predict(X)

            if diagnostic == True:
                print(typ, gp.kernel_)

            return np.sum((mean_ntk - mean)**2)

    for depth in ntk_depth:
        # Kernels
        if noise != 0.0 and white == True and alpha == False:
            ntk = (
                ConstantKernel(
                    constant_value=1,
                    constant_value_bounds=c_val_bounds
                ) *
                NTK(depth=depth, c=2,
                    bias=1e-1,
                    bias_bounds=(1e-9, 1e3)
                ) + 
                WhiteKernel(
                    noise_level=0.15**2, 
                    noise_level_bounds='fixed' if w_fixed else (1e-4, 0.5))#(0.001, 1))
            )
        else:
            ntk = (
                ConstantKernel(
                    constant_value=1,
                    constant_value_bounds=c_val_bounds#(1e-9, 1e2)
                ) *
                NTK(depth=depth, c=2,
                    bias=1e-1,
                    bias_bounds=(1e-9, 1e3)
                )
            )

        gp_ntk = GPR(kernel=ntk, alpha=1e-9 if not alpha else 0.15**2, normalize_y=True,
                    n_restarts_optimizer=9, random_state=3480795)
        gp_ntk.fit(X_train, y_train)
        print(f'NTK {depth} : ', gp_ntk.kernel_)
        mean_ntk = gp_ntk.predict(X)
        
        # print(gp_ntk.kernel_.get_params())


        if noise != 0.0 and white == True:
            lpk = (
                ConstantKernel(
                    constant_value=gp_ntk.kernel_.get_params()['k1__k1__constant_value'],#1**2,#0.224**2,
                    constant_value_bounds='fixed'#c_val_bounds
                ) *
                Matern(
                    nu=1/2,
                    length_scale=np.array([np.random.rand()*np.random.randint(1, 1000) 
                                        for i in range(0, X.shape[1])]) if ansiotropic else 1,
                    length_scale_bounds='fixed'  # (1e-1, 1e3),
                ) + 
                WhiteKernel(
                    noise_level=gp_ntk.kernel_.get_params()['k2__noise_level'], 
                    noise_level_bounds='fixed')
            )
        else:
            lpk = (
                ConstantKernel(
                    constant_value=gp_ntk.kernel_.get_params()['k1__constant_value'],#1**2,#0.224**2,
                    constant_value_bounds='fixed'#c_val_bounds
                ) *
                Matern(
                    nu=1/2,
                    # anisotropic true -> trying to fit product of laplace kernels with different lengthscales
                    length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                        for i in range(0, X.shape[1])]) if ansiotropic else 1,
                    # length_scale=1,
                    length_scale_bounds='fixed'  # (1e-1, 1e3),
                )
            )

        gp_lpk = GPR(kernel=lpk, alpha=1e-9 if not alpha else 0.15**2, normalize_y=True,
                    n_restarts_optimizer=0, random_state=3480795)

        if ansiotropic:
            ell_lpk = optimize.minimize(g, np.array([np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]), args=(
                gp_lpk, mean_ntk, f'LPK {depth}:', corrcoef_type), bounds=tuple((0.00001, 10) for n in range(0, X.shape[1])))
            for i in range(0, opt_resets):
                tmp = optimize.minimize(g, np.array([[np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]]), args=(
                    gp_lpk, mean_ntk, f'LPK {depth}:', corrcoef_type), bounds=tuple((0.00001, 10**i) for n in range(0, X.shape[1])))
                if tmp.fun < ell_lpk.fun:
                    ell_lpk = tmp

        else:
            ell_lpk = optimize.minimize_scalar(g, args=(
                gp_lpk, mean_ntk, f'LPK {depth}:', corrcoef_type), method='bounded', bounds=opt_bounds, options={'maxiter': 10000})
            for i in range(0, opt_resets):
                tmp = optimize.minimize_scalar(g, args=(gp_lpk, mean_ntk, f'LPK {depth}:', corrcoef_type),
                                               method='bounded', bounds=[opt_lower, opt_upper*(10**i)], options={'maxiter': 10000})
                if tmp.fun < ell_lpk.fun:
                    ell_lpk = tmp

        try:
            gp_lpk.set_params(**{'kernel__k2__length_scale': ell_lpk.x})
        except:
            gp_lpk.set_params(**{'kernel__k1__k2__length_scale': ell_lpk.x})
        gp_lpk.fit(X_train, y_train)
        mean_lpk_opt = gp_lpk.predict(X)



        # Begin gathering return values
        return_vals[f'lpk_{depth}'] = ell_lpk.x
        return_vals[f'f_lpk_{depth}'] = -ell_lpk.fun

        if corrcoef_type == 's':
            return_vals[f'fp_lpk_{depth}'] = np.corrcoef((mean_ntk)[:,0], (mean_lpk_opt)[:,0])[0, 1]
            corr_lpk = return_vals[f'fp_lpk_{depth}']
        else:
            return_vals[f'fs_lpk_{depth}'] = spearmanr(mean_ntk, mean_lpk_opt)[0]
            corr_lpk = return_vals[f'fs_lpk_{depth}']



        if norm:
            if noise != 0.0 and white == True and alpha == False:
                rbf = (
                    ConstantKernel(
                        constant_value=1, 
                        constant_value_bounds=c_val_bounds
                    ) * 
                    Matern(
                        nu=np.inf,
                        length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                               for i in range(0, X.shape[1])]) if ansiotropic else 1,
                        length_scale_bounds='fixed'#(1e-1, 1e3), False
                    ) + 
                    WhiteKernel(
                        noise_level=gp_ntk.kernel_.get_params()['k2__noise_level'],
                        noise_level_bounds='fixed'
                    )
                )
            else:
                rbf = (
                    ConstantKernel(
                        constant_value=1,
                        constant_value_bounds=c_val_bounds
                    ) *
                    Matern(
                        nu=np.inf,
                        length_scale=np.array([np.random.rand()*np.random.randint(1, 1000)
                                               for i in range(0, X.shape[1])]) if ansiotropic else 1,
                        length_scale_bounds='fixed',  # (1e-1, 1e3),
                    )
                )

            gp_rbf = GPR(kernel=rbf, alpha=1e-9 if not alpha else 0.15**2, normalize_y=True, n_restarts_optimizer=0, random_state=3480795)

            if ansiotropic:
                ell_rbf = optimize.minimize(g, np.array([np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]), args=(
                    gp_rbf, mean_ntk, f'RBF {depth}:', corrcoef_type), bounds=tuple((0.00001, 10) for n in range(0, X.shape[1])))
                for i in range(0, opt_resets):
                    tmp = optimize.minimize(g, np.array([[np.random.rand()*np.random.randint(1, 1000) for i in range(0, X.shape[1])]]), args=(
                        gp_rbf, mean_ntk, f'RBF {depth}:', corrcoef_type), bounds=tuple((0.00001, 10**i) for n in range(0, X.shape[1])))
                    if tmp.fun < ell_rbf.fun:
                        ell_rbf = tmp

            else:
                ell_rbf = optimize.minimize_scalar(g, args=(
                    gp_rbf, mean_ntk, f'RBF {depth}:', corrcoef_type), method='bounded', bounds=opt_bounds, options={'maxiter': 10000})
                for i in range(0, opt_resets):
                    tmp = optimize.minimize_scalar(g, args=(gp_rbf, mean_ntk, f'RBF {depth}:', corrcoef_type),
                                                method='bounded', bounds=[opt_lower, opt_upper*(10**i)], options={'maxiter': 10000})
                    if tmp.fun < ell_rbf.fun:
                        ell_rbf = tmp

            try:
                gp_rbf.set_params(**{'kernel__k2__length_scale': ell_rbf.x})
            except:
                gp_rbf.set_params(**{'kernel__k1__k2__length_scale': ell_rbf.x})
            gp_rbf.fit(X_train, y_train)
            mean_rbf_opt = gp_rbf.predict(X)

            # Continue gathering return values
            return_vals[f'rbf_{depth}'] = ell_rbf.x
            return_vals[f'f_rbf_{depth}'] = -ell_rbf.fun

            if corrcoef_type == 's':
                return_vals[f'fp_rbf_{depth}'] = np.corrcoef((mean_ntk)[:,0], (mean_rbf_opt)[:,0])[0, 1]
                corr_rbf = return_vals[f'fp_rbf_{depth}']
            else:
                return_vals[f'fs_rbf_{depth}'] = spearmanr(mean_ntk, mean_rbf_opt)[0]
                corr_rbf = return_vals[f'fs_rbf_{depth}']



        if figs:
            print("Creating plots...")
            plt.ioff()
            rvr = os.path.join(exp_name, 'res_vs_res')
            rvy = os.path.join(exp_name, 'res_vs_y')
            yvy = os.path.join(exp_name, 'y_vs_y')
            if not os.path.isdir(rvr):
                os.makedirs(rvr)
            if not os.path.isdir(rvy):
                os.makedirs(rvy)
            if not os.path.isdir(yvy):
                os.makedirs(yvy)
            # residuals vs y_hat (LPK and NTK SEPARATE)
            fig, axes = plt.subplots(nrows=1, ncols=3 if norm else 2, sharex=True,sharey=True, figsize=(12 if norm else 8, 4))
            axes[0].scatter(y-mean_ntk, mean_ntk)
            axes[0].set_title('NTK')
            axes[1].scatter(y-mean_lpk_opt, mean_lpk_opt)
            axes[1].set_title('LPK')
            title = f"{gp_ntk.kernel_}\n{gp_lpk.kernel_}"
            if norm:
                axes[2].scatter(y-mean_rbf_opt, mean_rbf_opt)
                axes[2].set_title('RBF')
                title += f"\n{gp_rbf.kernel_}"
            fig.supxlabel('Residuals')
            fig.supylabel(r'$\hat{y}$')
            fig.suptitle(r"Residuals vs. $\hat{y}$ for " + f"NTK Depth = {depth}\n" + title)
            fig.tight_layout()
            fig.savefig(f'{rvy}/depth_{depth}_{job_name}.png', dpi=150, transparent=False, bbox_inches='tight')
            
            # residuals vs residuals (LPK vs NTK)
            fig, axes = plt.subplots(nrows=1, ncols=2 if norm else 1, sharex=True, sharey=True, figsize=(8 if norm else 4, 4))
            if norm:
                axes[0].scatter(y-mean_lpk_opt, y-mean_ntk)
                axes[0].set_xlabel(f"LPK Residuals\nCorr: {round(corr_lpk, 4)}")
                axes[1].scatter(y-mean_rbf_opt, y-mean_ntk)
                axes[1].set_xlabel(f"RBF Residuals\nCorr: {round(corr_rbf, 4)}")
            else:
                axes.scatter(y-mean_lpk_opt, y-mean_ntk)
                axes.set_xlabel(f"LPK Residuals\nCorr: {round(corr_lpk, 4)}")
            fig.supylabel("NTK Residuals")
            fig.suptitle(f"Residuals vs. Residuals for NTK Depth = {depth}\n" + title)
            fig.tight_layout()
            fig.savefig(f'{rvr}/depth_{depth}_{job_name}.png', dpi=150, transparent=False, bbox_inches='tight')

            # y_hat vs y_hat (LPK vs NTK)
            fig, axes = plt.subplots(nrows=1, ncols=2 if norm else 1, sharex=True, sharey=True, figsize=(8 if norm else 4, 4))
            if norm:
                axes[0].scatter(mean_lpk_opt, mean_ntk)
                axes[0].set_xlabel(r'$y_{LPK}$')
                axes[1].scatter(mean_rbf_opt, mean_ntk)
                axes[1].set_xlabel(r'$y_{RBF}$')
            else:
                axes.scatter(mean_lpk_opt, mean_ntk)
                axes.set_xlabel(r'$y_{LPK}$')
            fig.supylabel(r'$y_{NTK}$')
            fig.suptitle(f"Predicted mean comparison for NTK Depth = {depth}\n" + title)
            fig.tight_layout()
            fig.savefig(f'{yvy}/depth_{depth}_{job_name}.png', dpi=150, transparent=False, bbox_inches='tight')
            plt.ion()

    return return_vals

In [3]:
# %%capture
# fried_num, 
# noise, 
# norm,
# c_val_bounds, 
# ntk_depth, 
# ansiotropic,
# white,
# w_fixed=False,
# alpha=False,
# opt_bounds = [0.0001, 10],
# opt_resets = 10,
# figs = False,
# diagnostic = False,
# exp_name = ''
depths=(3,100)
params = [
    [1, 0.15, True, (1e-5, 1e3)],
    # [2, 0.15, False, 'fixed', 'p']
]

kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': False, 'alpha': True, 'w_fixed': True, 'opt_resets': 10, 'diagnostic': False, 'figs': True, 'exp_name': 'diagnostic'}

test = []
for param in params:
    test.append(runner(*param, **kparams))

# with Pool(processes=cpu_count()) as p:
#     test = p.starmap(
#         partial(runner, **kparams), params
#     )

# test = runner(*[        1,   0.15, True,      (1e-9, 1e5),  100,           'p'], 
#     diagnostic=False, ansiotropic=True, white=False)

Friedman 1, Noise: 0.15, Norm: True, cv_bounds: (1e-05, 1000.0), white/bounds/alpha: (False, True, True)


/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


NTK 3 :  1.53**2 * NTK(depth=3, c=2.000, bias=0.408)


/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1e+03. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1e+03. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1e+03. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gau

Creating plots...
NTK 100 :  1.13**2 * NTK(depth=100, c=2.000, bias=0.000)


/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1e+03. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1e+03. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1e+03. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gau

Creating plots...


Bad pipe message: %s [b"\x12y^QR\xca\xbf\xe1W]\xe7\x8a[\r\x8aHv\xa0 \xf3\xe2\x7f\x87\xe23\x0cWg\xd0\x97\x96\xf5j\xe6:\xf6\xae@\xad\xd7'%\x92\xb2d:"]
Bad pipe message: %s [b"v\xf8\x8e\xe5\x89\x17\xcf \xf9\xac#\xa1Y,\x16\xa1\xab\xed\x00\x00\xf4\xc00\xc0,\xc0(\xc0$\xc0\x14\xc0\n\x00\xa5\x00\xa3\x00\xa1\x00\x9f\x00k\x00j\x00i\x00h\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19\x00\xa7\x00m\x00:\x00\x89\xc02\xc0.\xc0*\xc0&\xc0\x0f\xc0\x05\x00\x9d\x00=\x005\x00\x84\xc0/\xc0+\xc0'\xc0#\xc0\x13\xc0\t\x00\xa4\x00\xa2\x00\xa0\x00\x9e\x00g\x00@\x00?\x00>\x003\x002\x001\x000\x00\x9a\x00\x99\x00\x98\x00\x97\x00E\x00D\x00C\x00B\xc0\x18\x00\xa6\x00l\x004\x00\x9b\x00F\xc01\xc0-\xc0)\xc0%\xc0\x0e\xc0\x04\x00\x9c\x00<\x00/\x00\x96\x00A\x00\x07\xc0\x11\xc0\x07\xc0\x16\x00\x18\xc0\x0c\xc0\x02\x00\x05\x00\x04\xc0\x12\xc0"]
Bad pipe message: %s [b'\xf6`\xf3\xfct\xe7']
Bad pipe message: %s [b'\xae\xe1\xad\xa4\xd1yW\xcat+\xdcb\xc6\xf7\x8d\xf3\x9c\x95\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\

In [17]:
test

[{'Dataset': 'Friedman 1',
  'Noise': True,
  'Norm': True,
  'Constant Value Bounds': 'unfixed',
  'White/Bounds/Alpha': (False, True, True),
  'x_lpk_3': 2.488931294620113e+00,
  'f_lpk_3': 1.9757516500454821,
  'Pcorr_lpk_3_res': 0.9986121860556133,
  'Pcorr_lpk_3_y': 0.9996283039733059,
  'x_rbf_3': 5.67588775091024,
  'f_rbf_3': 215.96399914279925,
  'Pcorr_rbf_3_res': 0.8511764357073708,
  'Pcorr_rbf_3_y': 0.9438891411436396,
  'x_lpk_100': 0.12113343299367206,
  'f_lpk_100': 3.1148395819952652,
  'Pcorr_lpk_100_res': 0.9992296500026737,
  'Pcorr_lpk_100_y': 0.9991958489539943,
  'x_rbf_100': 194.41789167889138,
  'f_rbf_100': 652.7702799366725,
  'Pcorr_rbf_100_res': 0.7566808237822621,
  'Pcorr_rbf_100_y': 0.683598238070282}]

## Experiment 0: Constant Value Optimization

In [13]:
params = [
    #fried_num  noise   norm c_val_bounds  
    [        1,   0.0, False,      'fixed', 'p'], # 0
    [        1,   0.0, False,  (1e-9, 1e5), 'p'], # 1
    [        1,   0.0,  True,      'fixed', 'p'], # 2
    [        1,   0.0,  True,  (1e-9, 1e5), 'p'], # 3
    [        1,  0.15, False,      'fixed', 'p'], # 4 
    [        1,  0.15, False,  (1e-9, 1e5), 'p'], # 5
    [        1,  0.15,  True,      'fixed', 'p'], # 6
    [        1,  0.15,  True,  (1e-9, 1e5), 'p'], # 7
    
    [        1,   0.0, False,      'fixed', 's'], # 8
    [        1,   0.0, False,  (1e-9, 1e5), 's'], # 9
    [        1,   0.0,  True,      'fixed', 's'], # 10
    [        1,   0.0,  True,  (1e-9, 1e5), 's'], # 11
    [        1,  0.15, False,      'fixed', 's'], # 12
    [        1,  0.15, False,  (1e-9, 1e5), 's'], # 13
    [        1,  0.15,  True,      'fixed', 's'], # 14
    [        1,  0.15,  True,  (1e-9, 1e5), 's'], # 15


    [        2,   0.0, False,      'fixed', 'p'], # 16
    [        2,   0.0, False,  (1e-9, 1e5), 'p'], # 17
    [        2,   0.0,  True,      'fixed', 'p'], # 18
    [        2,   0.0,  True,  (1e-9, 1e5), 'p'], # 19
    [        2,  0.15, False,      'fixed', 'p'], # 20
    [        2,  0.15, False,  (1e-9, 1e5), 'p'], # 21
    [        2,  0.15,  True,      'fixed', 'p'], # 22
    [        2,  0.15,  True,  (1e-9, 1e5), 'p'], # 23

    [        2,   0.0, False,      'fixed', 's'], # 24
    [        2,   0.0, False,  (1e-9, 1e5), 's'], # 25
    [        2,   0.0,  True,      'fixed', 's'], # 26
    [        2,   0.0,  True,  (1e-9, 1e5), 's'], # 27
    [        2,  0.15, False,      'fixed', 's'], # 28
    [        2,  0.15, False,  (1e-9, 1e5), 's'], # 29
    [        2,  0.15,  True,      'fixed', 's'], # 30
    [        2,  0.15,  True,  (1e-9, 1e5), 's'], # 31


    [        3,   0.0, False,      'fixed', 'p'], # 32
    [        3,   0.0, False,  (1e-9, 1e5), 'p'], # 33
    [        3,   0.0,  True,      'fixed', 'p'], # 34
    [        3,   0.0,  True,  (1e-9, 1e5), 'p'], # 35
    [        3,  0.15, False,      'fixed', 'p'], # 36
    [        3,  0.15, False,  (1e-9, 1e5), 'p'], # 37
    [        3,  0.15,  True,      'fixed', 'p'], # 38
    [        3,  0.15,  True,  (1e-9, 1e5), 'p'], # 39
    
    [        3,   0.0, False,      'fixed', 's'], # 40
    [        3,   0.0, False,  (1e-9, 1e5), 's'], # 41
    [        3,   0.0,  True,      'fixed', 's'], # 42
    [        3,   0.0,  True,  (1e-9, 1e5), 's'], # 43
    [        3,  0.15, False,      'fixed', 's'], # 44
    [        3,  0.15, False,  (1e-9, 1e5), 's'], # 45
    [        3,  0.15,  True,      'fixed', 's'], # 46
    [        3,  0.15,  True,  (1e-9, 1e5), 's'], # 47
]


In [32]:
params = [
    #fried_num  noise   norm c_val_bounds  
    [        1,   0.0, False,      'fixed'], # 0
    [        1,   0.0, False,  (1e-9, 1e5)], # 1
    [        1,   0.0,  True,      'fixed'], # 2
    [        1,   0.0,  True,  (1e-9, 1e5)], # 3
    [        1,  0.15, False,      'fixed'], # 4 
    [        1,  0.15, False,  (1e-9, 1e5)], # 5
    [        1,  0.15,  True,      'fixed'], # 6
    [        1,  0.15,  True,  (1e-9, 1e5)], # 7
    
    [        1,   0.0, False,      'fixed'], # 8
    [        1,   0.0, False,  (1e-9, 1e5)], # 9
    [        1,   0.0,  True,      'fixed'], # 10
    [        1,   0.0,  True,  (1e-9, 1e5)], # 11
    [        1,  0.15, False,      'fixed'], # 12
    [        1,  0.15, False,  (1e-9, 1e5)], # 13
    [        1,  0.15,  True,      'fixed'], # 14
    [        1,  0.15,  True,  (1e-9, 1e5)], # 15


    [        2,   0.0, False,      'fixed'], # 16
    [        2,   0.0, False,  (1e-9, 1e5)], # 17
    [        2,   0.0,  True,      'fixed'], # 18
    [        2,   0.0,  True,  (1e-9, 1e5)], # 19
    [        2,  0.15, False,      'fixed'], # 20
    [        2,  0.15, False,  (1e-9, 1e5)], # 21
    [        2,  0.15,  True,      'fixed'], # 22
    [        2,  0.15,  True,  (1e-9, 1e5)], # 23

    [        2,   0.0, False,      'fixed'], # 24
    [        2,   0.0, False,  (1e-9, 1e5)], # 25
    [        2,   0.0,  True,      'fixed'], # 26
    [        2,   0.0,  True,  (1e-9, 1e5)], # 27
    [        2,  0.15, False,      'fixed'], # 28
    [        2,  0.15, False,  (1e-9, 1e5)], # 29
    [        2,  0.15,  True,      'fixed'], # 30
    [        2,  0.15,  True,  (1e-9, 1e5)], # 31


    [        3,   0.0, False,      'fixed'], # 32
    [        3,   0.0, False,  (1e-9, 1e5)], # 33
    [        3,   0.0,  True,      'fixed'], # 34
    [        3,   0.0,  True,  (1e-9, 1e5)], # 35
    [        3,  0.15, False,      'fixed'], # 36
    [        3,  0.15, False,  (1e-9, 1e5)], # 37
    [        3,  0.15,  True,      'fixed'], # 38
    [        3,  0.15,  True,  (1e-9, 1e5)], # 39
    
    [        3,   0.0, False,      'fixed'], # 40
    [        3,   0.0, False,  (1e-9, 1e5)], # 41
    [        3,   0.0,  True,      'fixed'], # 42
    [        3,   0.0,  True,  (1e-9, 1e5)], # 43
    [        3,  0.15, False,      'fixed'], # 44
    [        3,  0.15, False,  (1e-9, 1e5)], # 45
    [        3,  0.15,  True,      'fixed'], # 46
    [        3,  0.15,  True,  (1e-9, 1e5)], # 47
]


In [33]:
arrays_lpk = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True],  # Noise
    [False, True],  # Norm
    depths,
    ['fixed', 'unfixed']
]

arrays_rbf = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True],  # Noise
    depths,
    ['fixed', 'unfixed']
]

index_lpk = pd.MultiIndex.from_product(
    arrays_lpk, names=["Dataset", "Noise", "Norm", "Depth", "Constant Value Bounds"])

index_rbf = pd.MultiIndex.from_product(
    arrays_rbf, names=["Dataset", "Noise", "Depth", "Constant Value Bounds"])


df_lpk = pd.DataFrame(
    index=index_lpk,
    columns=['X', 'f(X)', 'Residual Correlation', 'Prediction Correlation'])

df_rbf = pd.DataFrame(
    index=index_rbf,
    columns=['X', 'f(X)', 'Residual Correlation', 'Prediction Correlation'])


In [34]:
%%capture
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': False, 'w_fixed': False, 'alpha': False, 'figs': True, 'exp_name': 'const_val_optimization'}
results = []
for param in params:
    results.append(runner(*param, **kparams))

In [35]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), "X"] = out['x_lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), "f(X)"] = out['f_lpk_' + str(depth)]

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), "Residual Correlation"] = out[f'Pcorr_lpk_{depth}_res']

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), "Prediction Correlation"] = out[f'Pcorr_lpk_{depth}_y']

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), "X"] = out['x_rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), "f(X)"] = out['f_rbf_' + str(depth)]

            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), "Residual Correlation"] = out[f'Pcorr_rbf_{depth}_res']
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), "Prediction Correlation"] = out[f'Pcorr_rbf_{depth}_y']

In [36]:
df_lpk_const, df_rbf_const = df_lpk, df_rbf
df_lpk_const.to_csv('./experiments/const_val_optimization/const_lpk_df.csv')
df_rbf_const.to_csv('./experiments/const_val_optimization/const_rbf_df.csv')

In [37]:
df_lpk_const

X  \
Dataset    Noise Norm  Depth Constant Value Bounds                      
Friedman 1 False False 3     fixed                           2.674982   
                             unfixed                         2.618161   
                       100   fixed                           0.227176   
                             unfixed                         0.228436   
                 True  3     fixed                           4.466817   
                             unfixed                         3.817673   
                       100   fixed                           0.121655   
                             unfixed                         0.121655   
           True  False 3     fixed                           2.684592   
                             unfixed                         2.548584   
                       100   fixed                           0.227071   
                             unfixed                         0.230078   
                 True  3     fixed                           4.451779   
                             unfixed                         3.712559   
                       100   fixed                           0.121555   
                             unfixed                         0.121555   
Friedman 2 False False 3     fixed                          39.036726   
                             unfixed                        28.078119   
                       100   fixed                          39.598555   
                             unfixed                        36.642396   
                 True  3     fixed                         414.954157   
                             unfixed                         1.451539   
                       100   fixed                           0.079023   
                             unfixed                         0.078974   
           True  False 3     fixed                          39.050165   
                             unfixed                        39.014538   
                       100   fixed                          39.615395   
                             unfixed                        35.777843   
                 True  3     fixed                         414.776216   
                             unfixed                         1.451548   
                       100   fixed                           0.078999   
                             unfixed                          0.07895   
Friedman 3 False False 3     fixed                  2830679109.167375   
                             unfixed                    525122.463032   
                       100   fixed                  3246700533.258884   
                             unfixed                4491409029.666467   
                 True  3     fixed                           476.8902   
                             unfixed                           1.4332   
                       100   fixed                           0.080185   
                             unfixed                         0.080185   
           True  False 3     fixed                           66.77035   
                             unfixed                        70.607198   
                       100   fixed                          60.604221   
                             unfixed                         59.45566   
                 True  3     fixed                         110.245187   
                             unfixed                         1.381298   
                       100   fixed                           0.075978   
                             unfixed                         0.075978   

                                                              f(X)  \
Dataset    Noise Norm  Depth Constant Value Bounds                   
Friedman 1 False False 3     fixed                       27.242726   
                             unfixed                     27.160187   
                       100   fixed                        2.602929   
                             unfixed                      2.450797 

In [38]:
df_rbf_const

X            f(X)  \
Dataset    Noise Depth Constant Value Bounds                                 
Friedman 1 False 3     fixed                      0.339907       59.415755   
                       unfixed                    0.335282       61.300654   
                 100   fixed                      0.141878        6.733503   
                       unfixed                    0.141877        6.733503   
           True  3     fixed                      0.338896       62.493316   
                       unfixed                    0.333737       64.651882   
                 100   fixed                      0.141478         6.96827   
                       unfixed                    0.141478         6.96827   
Friedman 2 False 3     fixed                       8.31451  7102919.349349   
                       unfixed                  288.616222  6851393.613491   
                 100   fixed                      9.882619  7075243.829898   
                       unfixed                  249.359564  6942249.030861   
           True  3     fixed                      8.318433  7103497.249455   
                       unfixed                  234.845432  6957225.808338   
                 100   fixed                      9.888335  7075791.914174   
                       unfixed                   76.443575  6902634.846604   
Friedman 3 False 3     fixed                    160.156354        1.749108   
                       unfixed                50822.488341        1.699798   
                 100   fixed                    155.352772        1.704467   
                       unfixed                43835.063246        1.639245   
           True  3     fixed                    173.250579        4.170882   
                       unfixed                68935.226673        4.167752   
                 100   fixed                    169.971003        4.031656   
                       unfixed                45527.220297        4.018503   

                                             Residual Correlation  \
Dataset    Noise Depth Constant Value Bounds                        
Friedman 1 False 3     fixed                             0.967631   
                       unfixed                           0.966854   
                 100   fixed                             0.997718   
                       unfixed                           0.997718   
           True  3     fixed                             0.965972   
                       unfixed                           0.965065   
                 100   fixed                             0.997628   
                       unfixed                           0.997628   
Friedman 2 False 3     fixed                             0.583468   
                       unfixed                           0.596426   
                 100   fixed                              0.58596   
                       unfixed                           0.595841   
           True  3     fixed                             0.583409   
                       unfixed                           0.597622   
                 100   fixed                             0.585902   
                       unfixed                           0.586379   
Friedman 3 False 3     fixed                             0.818458   
                       unfixed                           0.823198   
                 100   fixed                             0.821116   
                       unfixed                           0.826213   
           True  3     fixed                             0.647543   
                       unfixed                           0.659225   
                 100   fixed                             0.659049   
                       unfixed                           0.651381   

                                             Prediction Correlation  
Dataset    Noise Depth Constant Value Bounds                         
Friedman 1 False 3     fixed                               0.986227  
                       

## Experiment 1, 1.5 - Noise optimization

In [20]:
params = [
    #fried_num  noise   norm c_val_bounds  
    [        1,  0.15, False,      'fixed', 'p'], # 4 
    [        1,  0.15, False,  (1e-9, 1e5), 'p'], # 5
    [        1,  0.15,  True,      'fixed', 'p'], # 6
    [        1,  0.15,  True,  (1e-9, 1e5), 'p'], # 7
    
    [        1,  0.15, False,      'fixed', 's'], # 12
    [        1,  0.15, False,  (1e-9, 1e5), 's'], # 13
    [        1,  0.15,  True,      'fixed', 's'], # 14
    [        1,  0.15,  True,  (1e-9, 1e5), 's'], # 15


    [        2,  0.15, False,      'fixed', 'p'], # 20
    [        2,  0.15, False,  (1e-9, 1e5), 'p'], # 21
    [        2,  0.15,  True,      'fixed', 'p'], # 22
    [        2,  0.15,  True,  (1e-9, 1e5), 'p'], # 23

    [        2,  0.15, False,      'fixed', 's'], # 28
    [        2,  0.15, False,  (1e-9, 1e5), 's'], # 29
    [        2,  0.15,  True,      'fixed', 's'], # 30
    [        2,  0.15,  True,  (1e-9, 1e5), 's'], # 31


    [        3,  0.15, False,      'fixed', 'p'], # 36
    [        3,  0.15, False,  (1e-9, 1e5), 'p'], # 37
    [        3,  0.15,  True,      'fixed', 'p'], # 38
    [        3,  0.15,  True,  (1e-9, 1e5), 'p'], # 39
    
    [        3,  0.15, False,      'fixed', 's'], # 44
    [        3,  0.15, False,  (1e-9, 1e5), 's'], # 45
    [        3,  0.15,  True,      'fixed', 's'], # 46
    [        3,  0.15,  True,  (1e-9, 1e5), 's'], # 47
]

In [21]:
arrays_lpk = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [True],  # Noise
    [False, True],  # Norm
    depths,
    ['fixed', 'unfixed']
]

arrays_rbf = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [True],  # Noise
    depths,
    ['fixed', 'unfixed']
]

index_lpk = pd.MultiIndex.from_product(
    arrays_lpk, names=["Dataset", "Noise", "Norm", "Depth", "Constant Value Bounds"])

index_rbf = pd.MultiIndex.from_product(
    arrays_rbf, names=["Dataset", "Noise", "Depth", "Constant Value Bounds"])


df_lpk = pd.DataFrame(
    index=index_lpk,
    columns=['Pearson X', 'Spearman X', 'Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])

df_rbf = pd.DataFrame(
    index=index_rbf,
    columns=['Pearson X', 'Spearman X', 'Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])


Pt. 1: No white kernel

In [22]:
%%capture
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': False, 'figs': True, 'exp_name': 'white_no'}
results = []
for param in params:
    results.append(runner(*param, **kparams))

In [23]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_lpk_' + str(depth)]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_lpk_' + str(depth)]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_lpk_' + str(depth)]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_rbf_' + str(depth)]

            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_rbf_' + str(depth)]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_rbf_' + str(depth)]

In [24]:
df_lpk_no_noise, df_rbf_no_noise = df_lpk, df_rbf
df_lpk_no_noise.to_csv('./experiments/white_no/white_no_lpk_df.csv')
df_rbf_no_noise.to_csv('./experiments/white_no/white_no_rbf_df.csv')

In [25]:
df_lpk_no_noise

Pearson X  \
Dataset    Noise Norm  Depth Constant Value Bounds                 
Friedman 1 True  False 3     fixed                      2.293013   
                             unfixed                    2.293012   
                       100   fixed                      0.233795   
                             unfixed                    0.233795   
                 True  3     fixed                       4.49483   
                             unfixed                    4.494827   
                       100   fixed                       0.11966   
                             unfixed                     0.11966   
Friedman 2 True  False 3     fixed                     30.498404   
                             unfixed                   30.498404   
                       100   fixed                      28.21029   
                             unfixed                    28.21029   
                 True  3     fixed                     21.458308   
                             unfixed                 30144.33585   
                       100   fixed                      0.078163   
                             unfixed                    0.078164   
Friedman 3 True  False 3     fixed                     76.177132   
                             unfixed                   76.177132   
                       100   fixed                     55.908955   
                             unfixed                   55.908954   
                 True  3     fixed                     13.000966   
                             unfixed                11915.886674   
                       100   fixed                      0.075137   
                             unfixed                    0.075137   

                                                          Spearman X  \
Dataset    Noise Norm  Depth Constant Value Bounds                     
Friedman 1 True  False 3     fixed                   15459141.187324   
                             unfixed                  38292948.67977   
                       100   fixed                          0.259339   
                             unfixed                        0.259339   
                 True  3     fixed                          5.801071   
                             unfixed                        5.801071   
                       100   fixed                  607323495.583559   
                             unfixed                         2.36076   
Friedman 2 True  False 3     fixed                          32.19287   
                             unfixed                        32.19287   
                       100   fixed                          9.442728   
                             unfixed                        9.442728   
                 True  3     fixed                          6.180382   
                             unfixed                       61.822705   
                       100   fixed                          1.459069   
                             unfixed                       38.194693   
Friedman 3 True  False 3     fixed                         32.728439   
                             unfixed                       32.728439   
                       100   fixed                         32.623864   
                             unfixed                       32.623864   
                 True  3     fixed                          9.999995   
                             unfixed                        39.28504   
                       100   fixed                        854.102004   
                             unfixed                     5760.832108   

                                                   Pearson f(X)  \
Dataset    Noise Norm  Depth Constant Value Bounds                
Friedman 1 True  False 3     fixed                     0.886163   
                             unfixed                   0.886163   
                       100   fixed                     0.995457   
                             unfixed                   0.995457   
           

In [26]:
df_rbf_no_noise

Pearson X      Spearman X  \
Dataset    Noise Depth Constant Value Bounds                                   
Friedman 1 True  3     fixed                        0.333082        0.340142   
                       unfixed                      0.333082        0.340142   
                 100   fixed                        0.133935        0.148435   
                       unfixed                      0.133935        0.148435   
Friedman 2 True  3     fixed                       71.092862       39.400864   
                       unfixed                  37890.882501    53631.235075   
                 100   fixed                       74.904212       39.406189   
                       unfixed                    763.931974   381966.506523   
Friedman 3 True  3     fixed                      709.437272      243.276637   
                       unfixed                 381970.002972  2439071.354276   
                 100   fixed                      680.902311     1567.715237   
                       unfixed                2605981.073063  2327363.049454   

                                             Pearson f(X) Spearman f(Xp)  \
Dataset    Noise Depth Constant Value Bounds                               
Friedman 1 True  3     fixed                     0.965526       0.912703   
                       unfixed                   0.965526       0.912703   
                 100   fixed                     0.997726       0.949499   
                       unfixed                   0.997726       0.949499   
Friedman 2 True  3     fixed                     0.585987        0.66291   
                       unfixed                   0.596987       0.672679   
                 100   fixed                      0.58997       0.663714   
                       unfixed                   0.596931       0.656406   
Friedman 3 True  3     fixed                     0.671422       0.721224   
                       unfixed                   0.683505       0.725617   
                 100   fixed                     0.677441       0.724596   
                       unfixed                    0.69447       0.750219   

                                             Spearman f(X) Pearson f(Xs)  
Dataset    Noise Depth Constant Value Bounds                              
Friedman 1 True  3     fixed                      0.913507      0.965324  
                       unfixed                    0.913507      0.965324  
                 100   fixed                      0.964956      0.997304  
                       unfixed                    0.964956      0.997304  
Friedman 2 True  3     fixed                      0.669091      0.584989  
                       unfixed                     0.68018      0.594913  
                 100   fixed                      0.668083      0.588752  
                       unfixed                     0.67532      0.588319  
Friedman 3 True  3     fixed                      0.727537      0.668973  
                       unfixed                    0.740702      0.678046  
                 100   fixed                      0.730021      0.675244  
                       unfixed                    0.751575      0.685944

Pt. 2: White Noise but FIXED

In [27]:
%%capture
depths = (3, 100)
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': True, 'w_fixed': True, 'figs': True, 'exp_name': 'white_fixed'}
results = []
for param in params:
    results.append(runner(*param, **kparams))

In [28]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_lpk_' + str(depth)]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_lpk_' + str(depth)]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_lpk_' + str(depth)]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_rbf_' + str(depth)]

            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_rbf_' + str(depth)]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_rbf_' + str(depth)]

In [29]:
df_lpk_noise_fixed = df_lpk
df_rbf_noise_fixed = df_rbf
df_lpk_noise_fixed.to_csv('./experiments/white_fixed/white_fixed_lpk_df.csv')
df_rbf_noise_fixed.to_csv('./experiments/white_fixed/white_fixed_rbf_df.csv')

By optimizing pearson we are not improving spearman.  Furthermore by optimizing spearman, we do not improve pearson... Mostly.

In [30]:
df_lpk_noise_fixed

Pearson X  \
Dataset    Noise Norm  Depth Constant Value Bounds                   
Friedman 1 True  False 3     fixed                        1.071639   
                             unfixed                      0.557225   
                       100   fixed                  8149611.127416   
                             unfixed                     61.376516   
                 True  3     fixed                        0.617496   
                             unfixed                      0.426124   
                       100   fixed                  7977825.794934   
                             unfixed                     76.748025   
Friedman 2 True  False 3     fixed                     3930.440934   
                             unfixed                      5.166595   
                       100   fixed                    55099.575605   
                             unfixed                      2.198148   
                 True  3     fixed                        0.117455   
                             unfixed                       1.84085   
                       100   fixed                        8.760134   
                             unfixed                      1.926519   
Friedman 3 True  False 3     fixed                      521.179961   
                             unfixed                    127.632576   
                       100   fixed                    82446.383041   
                             unfixed                      1.854608   
                 True  3     fixed                        0.166201   
                             unfixed                      0.388739   
                       100   fixed                        4.263131   
                             unfixed                      4.609808   

                                                      Spearman X Pearson f(X)  \
Dataset    Noise Norm  Depth Constant Value Bounds                              
Friedman 1 True  False 3     fixed                      1.403772     0.995711   
                             unfixed                    0.774402     0.996631   
                       100   fixed                     999.99997          1.0   
                             unfixed                    0.083209          1.0   
                 True  3     fixed                      0.688188     0.994468   
                             unfixed                    9.999995     0.998599   
                       100   fixed                     99.999992          1.0   
                             unfixed                    9.999995          1.0   
Friedman 2 True  False 3     fixed                   4304.411928      0.99976   
                             unfixed                   17.868999     0.996807   
                       100   fixed                  60557.282333     0.999996   
                             unfixed                    5.904216     0.999972   
                 True  3     fixed                      0.148691     0.992477   
                             unfixed                    9.999995     0.990891   
                       100   fixed                      11.05768     0.999995   
                             unfixed                    9.999995     0.999977   
Friedman 3 True  False 3     fixed                    733.137561     0.950669   
                             unfixed                  143.286634     0.958525   
                       100   fixed                  68091.655086     0.999999   
                             unfixed                     2.36076     0.999989   
                 True  3     fixed                      0.131661     0.928064   
                             unfixed                    3.093979     0.988041   
                       100   fixed                      3.877282     0.999979   
                             unfixed                    6.180382     0.999864   

                                                   Spearman f(Xp)  \
Dataset    Noise Norm  Depth Constant Value Bounds          

In [31]:
df_rbf_noise_fixed

Pearson X Spearman X  \
Dataset    Noise Depth Constant Value Bounds                           
Friedman 1 True  3     fixed                     0.480355   0.502029   
                       unfixed                    1.54945   9.999995   
                 100   fixed                  1997.665335   6.180382   
                       unfixed                    9.14625   9.999995   
Friedman 2 True  3     fixed                     0.087652   9.999995   
                       unfixed                   0.557636   9.999995   
                 100   fixed                     1.429003   9.999995   
                       unfixed                   0.602711   9.999995   
Friedman 3 True  3     fixed                     0.132203   0.083994   
                       unfixed                   0.108457   9.999995   
                 100   fixed                     1.004752   0.901786   
                       unfixed                   1.078394   9.999995   

                                             Pearson f(X) Spearman f(Xp)  \
Dataset    Noise Depth Constant Value Bounds                               
Friedman 1 True  3     fixed                     0.996606       0.995056   
                       unfixed                    0.99033       0.988755   
                 100   fixed                          1.0            1.0   
                       unfixed                        1.0            1.0   
Friedman 2 True  3     fixed                     0.993683        0.98333   
                       unfixed                   0.990237       0.988083   
                 100   fixed                     0.999989       0.999832   
                       unfixed                   0.999977       0.999796   
Friedman 3 True  3     fixed                     0.934161       0.947507   
                       unfixed                    0.98526       0.989703   
                 100   fixed                     0.999966       0.999904   
                       unfixed                   0.999864        0.99958   

                                             Spearman f(X) Pearson f(Xs)  
Dataset    Noise Depth Constant Value Bounds                              
Friedman 1 True  3     fixed                      0.995236      0.996591  
                       unfixed                    0.988755       0.99033  
                 100   fixed                           1.0      0.999999  
                       unfixed                         1.0           1.0  
Friedman 2 True  3     fixed                      0.948431      0.974762  
                       unfixed                    0.948203       0.97474  
                 100   fixed                      0.999832      0.999977  
                       unfixed                    0.999796      0.999977  
Friedman 3 True  3     fixed                      0.951575      0.931831  
                       unfixed                    0.901638      0.886219  
                 100   fixed                      0.999952      0.999961  
                       unfixed                     0.99958      0.999864

Pt 3: White Noise but OPTIMIZED

In [32]:
%%capture
depths=(3, 100)
kparams = {'ntk_depth': depths, 'ansiotropic': False, 'white': True, 'w_fixed': False, 'figs': True, 'exp_name': 'white_opt'}
results = []
for param in params:
    results.append(runner(*param, **kparams))

In [33]:
for out in results:
    for depth in depths:

        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['lpk_' + str(depth)]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_lpk_' + str(depth)]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_lpk_' + str(depth)]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_lpk_' + str(depth)]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "X"] = out['rbf_' + str(depth)]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out["Constant Value Bounds"]), out['Corr Type'] + "f(X)"] = out['f_rbf_' + str(depth)]

            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Spearman f(Xp)"] = out['fs_rbf_' + str(depth)]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            out["Constant Value Bounds"]), "Pearson f(Xs)"] = out['fp_rbf_' + str(depth)]

In [34]:
df_lpk_noise_optimized, df_rbf_noise_optimized = df_lpk, df_rbf
df_lpk_noise_optimized.to_csv('./experiments/white_opt/white_opt_lpk_df.csv')
df_rbf_noise_optimized.to_csv('./experiments/white_opt/white_opt_rbf_df.csv')

In [35]:
df_lpk_noise_optimized

Pearson X  \
Dataset    Noise Norm  Depth Constant Value Bounds                   
Friedman 1 True  False 3     fixed                        0.877504   
                             unfixed                      0.983673   
                       100   fixed                  4376125.040017   
                             unfixed                     66.986787   
                 True  3     fixed                        0.545422   
                             unfixed                      3.357961   
                       100   fixed                        0.254847   
                             unfixed                      0.115278   
Friedman 2 True  False 3     fixed                     1326.804385   
                             unfixed                  62634.061937   
                       100   fixed                    20970.215769   
                             unfixed                  62634.063081   
                 True  3     fixed                        0.107636   
                             unfixed                      3.247937   
                       100   fixed                        4.537584   
                             unfixed                      3.247937   
Friedman 3 True  False 3     fixed                      405.736359   
                             unfixed                    164.519408   
                       100   fixed                     75180.79484   
                             unfixed                  55519.450925   
                 True  3     fixed                        0.154682   
                             unfixed                      4.555568   
                       100   fixed                        0.133128   
                             unfixed                      7.782568   

                                                      Spearman X Pearson f(X)  \
Dataset    Noise Norm  Depth Constant Value Bounds                              
Friedman 1 True  False 3     fixed                      3.819726     0.735696   
                             unfixed                     0.76903     0.858571   
                       100   fixed                     999.99997          1.0   
                             unfixed                   99.999992          1.0   
                 True  3     fixed                      0.678438     0.767429   
                             unfixed                    9.999995     0.955343   
                       100   fixed                      0.118688     0.795826   
                             unfixed                    9.999995      0.99508   
Friedman 2 True  False 3     fixed                   1534.883494     0.976213   
                             unfixed                99999.998491     0.956178   
                       100   fixed                  18086.801399     0.999968   
                             unfixed                99999.998491     0.999811   
                 True  3     fixed                      0.135065     0.986655   
                             unfixed                    9.999995     0.961168   
                       100   fixed                      4.934267     0.999983   
                             unfixed                    9.999995      0.99992   
Friedman 3 True  False 3     fixed                    416.647604      0.78376   
                             unfixed                  196.601214     0.906679   
                       100   fixed                  76393.204371     0.999999   
                             unfixed                99999.998491     0.999987   
                 True  3     fixed                      0.058922     0.715744   
                             unfixed                    4.164141      0.98281   
                       100   fixed                      0.060661     0.983064   
                             unfixed                    6.180382     0.955475   

                                                   Spearman f(Xp)  \
Dataset    Noise Norm  Depth Constant Value Bounds          

In [36]:
df_rbf_noise_optimized

Pearson X Spearman X  \
Dataset    Noise Depth Constant Value Bounds                        
Friedman 1 True  3     fixed                  0.454959   0.438926   
                       unfixed                3.290237   9.999995   
                 100   fixed                  0.362754   3.819725   
                       unfixed                3.290237   9.999995   
Friedman 2 True  3     fixed                  0.077629   9.999995   
                       unfixed                 1.36002   9.999995   
                 100   fixed                  1.023712   9.999995   
                       unfixed                1.987786   9.999995   
Friedman 3 True  3     fixed                  0.150608   0.121749   
                       unfixed                2.420776   9.999995   
                 100   fixed                  0.104079   0.087628   
                       unfixed                2.420776   9.999995   

                                             Pearson f(X) Spearman f(Xp)  \
Dataset    Noise Depth Constant Value Bounds                               
Friedman 1 True  3     fixed                     0.781853       0.792199   
                       unfixed                   0.744623        0.75814   
                 100   fixed                     0.796797       0.918404   
                       unfixed                   0.779677       0.912115   
Friedman 2 True  3     fixed                     0.988599       0.972829   
                       unfixed                   0.961168       0.925245   
                 100   fixed                     0.999962        0.99976   
                       unfixed                    0.99992       0.999604   
Friedman 3 True  3     fixed                      0.72255       0.816814   
                       unfixed                   0.647686       0.768125   
                 100   fixed                     0.984606       0.972133   
                       unfixed                   0.955475       0.934929   

                                             Spearman f(X) Pearson f(Xs)  
Dataset    Noise Depth Constant Value Bounds                              
Friedman 1 True  3     fixed                      0.792547      0.781775  
                       unfixed                     0.75814      0.744623  
                 100   fixed                      0.912139      0.780317  
                       unfixed                    0.912115      0.779677  
Friedman 2 True  3     fixed                      0.925521      0.961194  
                       unfixed                    0.925245      0.961167  
                 100   fixed                       0.99964       0.99992  
                       unfixed                    0.999604       0.99992  
Friedman 3 True  3     fixed                      0.820534       0.72178  
                       unfixed                    0.768125      0.647686  
                 100   fixed                      0.972889      0.984367  
                       unfixed                    0.934929      0.955475

rbf lengthscale infinite -> linear (hyperplane)

## Experiment 2 - Ansiotropic LPK/RBF kernels
* Friedman 1 Noisy Norm
* Friedman 2 Noiseless Nonnormed

In [16]:
arrays_lpk = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True], # Noise
    [False, True], # Norm
    [3, 100], # Depths
]

arrays_rbf = [
    ['Friedman 1', 'Friedman 2', 'Friedman 3'],
    [False, True], # Noise
    [3, 100], # Depths
]

index_lpk = pd.MultiIndex.from_product(
    arrays_lpk, names=["Dataset", "Noise", "Norm", "Depth"])

index_rbf = pd.MultiIndex.from_product(
    arrays_rbf, names=["Dataset", "Noise", "Depth"])


df_lpk = pd.DataFrame(
    index=index_lpk,
    columns=['Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])

df_rbf = pd.DataFrame(
    index=index_rbf,
    columns=['Pearson f(X)', 'Spearman f(Xp)', 'Spearman f(X)', 'Pearson f(Xs)'])


In [17]:
df_lpk

Pearson f(X) Spearman f(Xp) Spearman f(X)  \
Dataset    Noise Norm  Depth                                             
Friedman 1 False False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
           True  False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
Friedman 2 False False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
           True  False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
Friedman 3 False False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
           True  False 3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   
                 True  3              NaN            NaN           NaN   
                       100            NaN            NaN           NaN   

                             Pearson f(Xs)  
Dataset    Noise Norm  Depth                
Friedman 1 False False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
           True  False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
Friedman 2 False False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
           True  False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
Friedman 3 False False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN  
           True  False 3               NaN  
                       100             NaN  
                 True  3               NaN  
                       100             NaN

In [18]:
% % capture
figs = []
params = [
    #    fried_num, noise,  norm, c_val_bounds, ntk_depth, corrcoef_type
    [1,   0.0, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 1
    [1,   0.0,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 3
    [1,  0.15, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 5
    [1,  0.15,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 7

    [1,   0.0, False,  (1e-9, 1e5),  [3, 100],           's'],  # 9
    [1,   0.0,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 11
    [1,  0.15, False,  (1e-9, 1e5),  [3, 100],           's'],  # 13
    [1,  0.15,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 15


    [2,   0.0, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 17
    [2,   0.0,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 19
    [2,  0.15, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 21
    [2,  0.15,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 23

    [2,   0.0, False,  (1e-9, 1e5),  [3, 100],           's'],  # 25
    [2,   0.0,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 27
    [2,  0.15, False,  (1e-9, 1e5),  [3, 100],           's'],  # 29
    [2,  0.15,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 31


    [3,   0.0, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 33
    [3,   0.0,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 35
    [3,  0.15, False,  (1e-9, 1e5),  [3, 100],           'p'],  # 37
    [3,  0.15,  True,  (1e-9, 1e5),  [3, 100],           'p'],  # 39

    [3,   0.0, False,  (1e-9, 1e5),  [3, 100],           's'],  # 41
    [3,   0.0,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 43
    [3,  0.15, False,  (1e-9, 1e5),  [3, 100],           's'],  # 45
    [3,  0.15,  True,  (1e-9, 1e5),  [3, 100],           's'],  # 47
]

for param in params:
    out = runner(*param, ansiotropic=True, figs=False)

    for depth in param[4]:
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out['Corr Type'] + "X"] = out['lpk_' + depth]
        df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                    out['Corr Type'] + "f(X)"] = out['f_lpk_' + depth]

        if out['Corr Type'] == 'Pearson ':
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        "Spearman f(Xp)"] = out['fs_lpk_' + depth]
        else:
            df_lpk.loc[(out["Dataset"], out["Noise"], out["Norm"],   depth,
                        "Pearson f(Xs)"] = out['fp_lpk_' + depth]

        if out["Norm"] == True:
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out['Corr Type'] + "X"] = out['rbf_' + depth]
            df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                        out['Corr Type'] + "f(X)"] = out['f_rbf_' + depth]
            if out['Corr Type'] == 'Pearson ':
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            "Spearman f(Xp)"] = out['fs_rbf_' + depth]
            else:
                df_rbf.loc[(out["Dataset"], out["Noise"],   depth,
                            "Pearson f(Xs)"] = out['fp_rbf_' + depth]


SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' on line 42 (1183362044.py, line 43)